In [1]:
#ref:- ILDC CJPE GITHUB

In [2]:
import pandas as pd

In [3]:
df = pd.read_csv("/media/anubhav/New Volume/vidhaan/dataset/data/ILDC_single/ILDC_single.csv")

In [4]:
NUM_CLASSES = 2
n_epochs = 20


In [5]:
train_set = df.query(" split=='train' ")
test_set = df.query(" split=='test' ")
val_set = df.query(" split=='dev' ")

In [6]:
del train_set['split'],train_set['name'],val_set['split'],val_set['name'],test_set['split'],test_set['name']

In [9]:
train_set.to_csv('/media/anubhav/New Volume/vidhaan/petition_acceptance/petition_data.csv',index=False)

In [6]:
train_set.head(10)

,text,label
0,"F. NARIMAN, J. Leave granted. In 2008, the Pu...",1
1,"S. THAKUR, J. Leave granted. These appeals ar...",0
2,"Markandey Katju, J. Leave granted. Heard lear...",1
3,"ALTAMAS KABIR,J. Leave granted. The question ...",1
4,"CIVIL APPEAL NO. 598 OF 2007 K. MATHUR, J. Th...",1
5,O R D E R CIVIL APPEAL NOS. 85-86 OF 2002 Thes...,0
6,O R D E R Arising out of S.L.P C No.3358 of ...,1
7,"SANTOSH HEGDE, J. Noticing certain companytra...",0
8,"2004 3 SCR 534 WITH W.P. C No. 852/91, C...",0
9,Arising out SLP c No. 16202 of 2003 With ...,1


In [7]:
x_train = train_set['text']
y_train = train_set['label']

x_test = test_set['text']
y_test = test_set['label']

x_val = val_set['text']
y_val = val_set['label']

In [8]:
del train_set,val_set,test_set

In [9]:
x_train.str.len()

0       28354
1       31637
2       18402
3       20133
4       28326
        ...  
5077    21900
5078     6574
5079     6998
5080     5042
5081     2871
Name: text, Length: 5082, dtype: int64

In [10]:
import tensorflow_hub as hub
import tensorflow_text as text

preprocessor = hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder-cmlm/multilingual-preprocess/2")
encoder = hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder-cmlm/multilingual-base/1")


def get_embeddings(sentences):
#   "return BERT-like embeddings of input text "
#   ""Args:
#     - sentences: list of strings
#   Output:
#     - BERT-like embeddings: tf.Tensor of shape=(len(sentences), 768)
#   ""
    preprocessed_text = preprocessor(sentences)
    return encoder(preprocessed_text)['pooled_output']


get_embeddings([
    "Questa collezione di Haiku è una porta aperta sulla cultura giapponese."]
)

<tf.Tensor: shape=(1, 768), dtype=float32, numpy=
array([[ 1.08269259e-01, -9.16296169e-02, -3.19652617e-01,
        -1.70691609e-02,  3.34133714e-01,  2.00266346e-01,
        -2.46935949e-01, -5.43310463e-01, -3.56957167e-01,
        -2.97796637e-01, -2.08980471e-01,  1.42026097e-01,
         2.72502378e-02, -4.33980823e-01, -5.73599935e-01,
        -8.82728934e-01,  2.37143964e-01, -1.87127888e-01,
        -2.56524235e-01, -2.46629849e-01,  6.32454276e-01,
         1.21353403e-01, -3.95606637e-01,  1.58425614e-01,
        -6.60170689e-02, -1.00859642e+00,  3.03424656e-01,
         2.40533184e-02, -9.84722450e-02, -6.36505723e-01,
         1.15401939e-01, -7.62529016e-01, -5.43886602e-01,
        -4.67391998e-01, -6.24043286e-01,  2.63592631e-01,
        -4.41221118e-01,  6.62037209e-02, -3.88788283e-01,
         1.61614940e-02,  1.15523271e-01, -4.35086548e-01,
        -5.12637436e-01,  2.74717063e-01, -3.44000697e-01,
        -1.88089922e-01,  1.52751924e-02, -3.23565662e-01,
      

In [11]:
# pip install tensorflow_hub tensorflow_text

In [14]:
from keras import backend as K
import tensorflow as tf

def balanced_recall(y_true, y_pred):
    """This function calculates the balanced recall metric
    recall = TP / (TP + FN)
    """
    recall_by_class = 0
    # iterate over each predicted class to get class-specific metric
    for i in range(y_pred.shape[1]):
        y_pred_class = y_pred[:, i]
        y_true_class = y_true[:, i]
        true_positives = K.sum(K.round(K.clip(y_true_class * y_pred_class, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true_class, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        recall_by_class = recall_by_class + recall
    return recall_by_class / y_pred.shape[1]

def balanced_precision(y_true, y_pred):
    """This function calculates the balanced precision metric
    precision = TP / (TP + FP)
    """
    precision_by_class = 0
    # iterate over each predicted class to get class-specific metric
    for i in range(y_pred.shape[1]):
        y_pred_class = y_pred[:, i]
        y_true_class = y_true[:, i]
        true_positives = K.sum(K.round(K.clip(y_true_class * y_pred_class, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred_class, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        precision_by_class = precision_by_class + precision
    # return average balanced metric for each class
    return precision_by_class / y_pred.shape[1]

def balanced_f1_score(y_true, y_pred):
    """This function calculates the F1 score metric"""
    precision = balanced_precision(y_true, y_pred)
    recall = balanced_recall(y_true, y_pred)
    return 2 * ((precision * recall) / (precision + recall + K.epsilon()))

In [37]:
i = tf.keras.layers.Input(shape=(x_train.shape[-2],x_train.shape[-1]), dtype=tf.string, name='text')
# i = tf.keras.layers.Input(shape=(x_train.shape[-2],x_train.shape[-1]),name='input',dtype=tf.string)
x = preprocessor(i)
x = encoder(x)
x = tf.keras.layers.Dropout(0.2, name="dropout")(x['pooled_output'])
x = tf.keras.layers.Dense(NUM_CLASSES, activation='softmax', name="output")(x)

model = tf.keras.Model(i, x)

IndexError: tuple index out of range

In [35]:
METRICS = [
      tf.keras.metrics.CategoricalAccuracy(name="accuracy"),
      balanced_recall,
      balanced_precision,
      balanced_f1_score
]

earlystop_callback = tf.keras.callbacks.EarlyStopping(monitor = "val_loss", 
                                                      patience = 3,
                                                      restore_best_weights = True)

model.compile(optimizer = "adam",
              loss = "sparse_categorical_crossentropy",
              metrics = METRICS)

model_fit = model.fit(x_train, 
                      y_train, 
                      epochs = n_epochs,
                      validation_data = (x_val, y_val),
                      callbacks = [earlystop_callback])

Epoch 1/20


ValueError: in user code:

    File "/home/anubhav/miniconda3/envs/vidhaan/lib/python3.7/site-packages/keras/engine/training.py", line 1051, in train_function  *
        return step_function(self, iterator)
    File "<ipython-input-14-8f1c946c95ef>", line 12, in balanced_recall  *
        y_true_class = y_true[:, i]

    ValueError: slice index 1 of dimension 1 out of bounds. for '{{node strided_slice_4}} = StridedSlice[Index=DT_INT32, T=DT_FLOAT, begin_mask=1, ellipsis_mask=0, end_mask=1, new_axis_mask=0, shrink_axis_mask=2](Cast_4, strided_slice_4/stack, strided_slice_4/stack_1, strided_slice_4/stack_2)' with input shapes: [?,1], [2], [2], [2] and with computed input tensors: input[1] = <0 1>, input[2] = <0 2>, input[3] = <1 1>.


In [20]:
x_train.shape

(5082,)

In [21]:
y_train.shape

(5082,)

In [22]:
x_val.shape

(994,)

In [23]:
y_val.shape

(994,)

In [24]:
x_test.shape

(1517,)

In [25]:
y_test.shape

(1517,)